In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.enableHiveSupport().master("local[2]").getOrCreate()

In [3]:
graph_schema = StructType([
    StructField("to", IntegerType(), False),
    StructField("from", IntegerType(), False)
])

In [4]:
dist_schema = StructType([
    StructField("vertex", IntegerType(), False),
    StructField("distance", IntegerType(), False)
])

In [5]:
def shortest_path(v_from, v_to, dataset_path=None):
    edges = spark.read.csv(dataset_path, sep="\t", schema=graph_schema)
    distanses = spark.createDataFrame(data=[], schema=dist_schema)
    
    edges_from = edges.select(col('to'), col('from').alias('id')) \
                .withColumn('dist', lit(1)) \
                .filter(col('to') == v_to) \
                .cache()

    distanses = edges_from.select(col('id').alias('vertex'), col('dist').alias('distance')).cache()
    
    count = 0
    tmp = edges_from
    itr = 1 

    while count == 0:
        tmp = tmp.join(edges.select(col('from'), col('to').alias('id')), on='id', how="inner") \
                 .join(distanses.withColumn("id", col("vertex")), on='id', how="left_outer") \
                 .withColumn("_dist", col('dist') + 1) \
                 .select(col('to'), col('from').alias('id'), col("_dist").alias('dist'))\
                 .cache()

        count = tmp.filter(col('id') == v_from).count()

        distanses = distanses.union(tmp.select(col('id'), col("dist"))).cache()

        print "itteration %s, count %s" % (itr, count)
        itr += 1
    
    return distanses.filter(col('vertex') == v_from).select(col('distance')).collect()[0].distance

In [6]:
print shortest_path(12, 34, "/data/twitter/twitter_sample2.txt")

itteration 1, count 0
itteration 2, count 0
itteration 3, count 0
itteration 4, count 0
itteration 5, count 0
itteration 6, count 0
itteration 7, count 1
8
